In [ ]:

import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/lexihenny/'
dir2='/Volumes/Extreme Pro 1/'
dir3='/Volumes/My Passport/'

In [ ]:
ds=xr.open_dataset(dir_data+'synoptic_geochars_era5tp_MAX.nc')
#ds=xr.open_dataset(dir2+'synoptic_geochars_era5tp.nc')
ivt_concat_era5tp=ds['ivt_concat']
iwv_concat_era5tp=ds['iwv_concat']
s_concat_era5tp=ds['s_concat']
mfc_concat_era5tp=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_era5reid_MAX.nc')
#ds=xr.open_dataset(dir2+'synoptic_geochars_era5reid.nc')
ivt_concat_era5reid=ds['ivt_concat']
iwv_concat_era5reid=ds['iwv_concat']
s_concat_era5reid=ds['s_concat']
mfc_concat_era5reid=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_era5mk_MAX.nc')
#ds=xr.open_dataset(dir2+'synoptic_geochars_era5mk.nc')
ivt_concat_era5mk=ds['ivt_concat']
iwv_concat_era5mk=ds['iwv_concat']
s_concat_era5mk=ds['s_concat']
mfc_concat_era5mk=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_merra2lo_MAX.nc')
#ds=xr.open_dataset(dir2+'synoptic_geochars_merra2lo.nc')
ivt_concat_merra2lo=ds['ivt_concat']
iwv_concat_merra2lo=ds['iwv_concat']
s_concat_merra2lo=ds['s_concat']
mfc_concat_merra2lo=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_merra2cndl_MAX.nc')
#ds=xr.open_dataset(dir2+'synoptic_geochars_merra2cndl.nc')
ivt_concat_merra2cndl=ds['ivt_concat']
iwv_concat_merra2cndl=ds['iwv_concat']
s_concat_merra2cndl=ds['s_concat']
mfc_concat_merra2cndl=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_merra2ac_MAX.nc')
#ds=xr.open_dataset(dir2+'synoptic_geochars_merra2ac.nc')
ivt_concat_merra2ac=ds['ivt_concat']
iwv_concat_merra2ac=ds['iwv_concat']
s_concat_merra2ac=ds['s_concat']
mfc_concat_merra2ac=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_jra55gw_MAX.nc')
#ds=xr.open_dataset(dir2+'synoptic_geochars_jra55gw.nc')
ivt_concat_jra55gw=ds['ivt_concat']
iwv_concat_jra55gw=ds['iwv_concat']
s_concat_jra55gw=ds['s_concat']

In [ ]:
lons_west=[x for x in ivt_concat_jra55gw.lon.values if x<-90]
lons_east=[x for x in ivt_concat_jra55gw.lon.values if x>=90]
sel_west=ivt_concat_jra55gw.sel(lon=lons_west)
sel_east=ivt_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,ivt_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5tp.lat.values,lon=ivt_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_ivt=interp.sel(lon=lons_center)


sel_west=iwv_concat_jra55gw.sel(lon=lons_west)
sel_east=iwv_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,iwv_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5tp.lat.values,lon=ivt_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_iwv=interp.sel(lon=lons_center)


sel_west=s_concat_jra55gw.sel(lon=lons_west)
sel_east=s_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,s_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5tp.lat.values,lon=ivt_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_s=interp.sel(lon=lons_center)

In [ ]:
ds=xr.open_dataset(dir_data+'synoptic_geotrends_arrays_artmip_MAX.nc')
#ds=xr.open_dataset(dir2+'synoptic_geotrends_arrays_artmip.nc')
slopes_array_ivt_era5tp=ds['slopes_array_ivt_era5tp']
slopes_array_ivt_era5reid=ds['slopes_array_ivt_era5reid']
slopes_array_ivt_era5mk=ds['slopes_array_ivt_era5mk']
slopes_array_ivt_merra2lo=ds['slopes_array_ivt_merra2lo']
slopes_array_ivt_merra2cndl=ds['slopes_array_ivt_merra2cndl']
slopes_array_ivt_merra2ac=ds['slopes_array_ivt_merra2ac']
slopes_array_ivt_jra55gw=ds['slopes_array_ivt_jra55gw']

sigs_array_ivt_era5tp=ds['sigs_array_ivt_era5tp']
sigs_array_ivt_era5reid=ds['sigs_array_ivt_era5reid']
sigs_array_ivt_era5mk=ds['sigs_array_ivt_era5mk']
sigs_array_ivt_merra2lo=ds['sigs_array_ivt_merra2lo']
sigs_array_ivt_merra2cndl=ds['sigs_array_ivt_merra2cndl']
sigs_array_ivt_merra2ac=ds['sigs_array_ivt_merra2ac']
sigs_array_ivt_jra55gw=ds['sigs_array_ivt_jra55gw']


slopes_array_iwv_era5tp=ds['slopes_array_iwv_era5tp']
slopes_array_iwv_era5reid=ds['slopes_array_iwv_era5reid']
slopes_array_iwv_era5mk=ds['slopes_array_iwv_era5mk']
slopes_array_iwv_merra2lo=ds['slopes_array_iwv_merra2lo']
slopes_array_iwv_merra2cndl=ds['slopes_array_iwv_merra2cndl']
slopes_array_iwv_merra2ac=ds['slopes_array_iwv_merra2ac']
slopes_array_iwv_jra55gw=ds['slopes_array_iwv_jra55gw']

sigs_array_iwv_era5tp=ds['sigs_array_iwv_era5tp']
sigs_array_iwv_era5reid=ds['sigs_array_iwv_era5reid']
sigs_array_iwv_era5mk=ds['sigs_array_iwv_era5mk']
sigs_array_iwv_merra2lo=ds['sigs_array_iwv_merra2lo']
sigs_array_iwv_merra2cndl=ds['sigs_array_iwv_merra2cndl']
sigs_array_iwv_merra2ac=ds['sigs_array_iwv_merra2ac']
sigs_array_iwv_jra55gw=ds['sigs_array_iwv_jra55gw']


slopes_array_s_era5tp=ds['slopes_array_s_era5tp']
slopes_array_s_era5reid=ds['slopes_array_s_era5reid']
slopes_array_s_era5mk=ds['slopes_array_s_era5mk']
slopes_array_s_merra2lo=ds['slopes_array_s_merra2lo']
slopes_array_s_merra2cndl=ds['slopes_array_s_merra2cndl']
slopes_array_s_merra2ac=ds['slopes_array_s_merra2ac']
slopes_array_s_jra55gw=ds['slopes_array_s_jra55gw']

sigs_array_s_era5tp=ds['sigs_array_s_era5tp']
sigs_array_s_era5reid=ds['sigs_array_s_era5reid']
sigs_array_s_era5mk=ds['sigs_array_s_era5mk']
sigs_array_s_merra2lo=ds['sigs_array_s_merra2lo']
sigs_array_s_merra2cndl=ds['sigs_array_s_merra2cndl']
sigs_array_s_merra2ac=ds['sigs_array_s_merra2ac']
sigs_array_s_jra55gw=ds['sigs_array_s_jra55gw']


slopes_array_mfc_era5tp=ds['slopes_array_mfc_era5tp']
slopes_array_mfc_era5reid=ds['slopes_array_mfc_era5reid']
slopes_array_mfc_era5mk=ds['slopes_array_mfc_era5mk']
slopes_array_mfc_merra2lo=ds['slopes_array_mfc_merra2lo']
slopes_array_mfc_merra2cndl=ds['slopes_array_mfc_merra2cndl']
slopes_array_mfc_merra2ac=ds['slopes_array_mfc_merra2ac']

sigs_array_mfc_era5tp=ds['sigs_array_mfc_era5tp']
sigs_array_mfc_era5reid=ds['sigs_array_mfc_era5reid']
sigs_array_mfc_era5mk=ds['sigs_array_mfc_era5mk']
sigs_array_mfc_merra2lo=ds['sigs_array_mfc_merra2lo']
sigs_array_mfc_merra2cndl=ds['sigs_array_mfc_merra2cndl']
sigs_array_mfc_merra2ac=ds['sigs_array_mfc_merra2ac']

In [ ]:
num_sets1=xr.ones_like(slopes_array_ivt_era5tp).where(slopes_array_ivt_era5tp>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_ivt_era5reid).where(slopes_array_ivt_era5reid>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_ivt_era5mk).where(slopes_array_ivt_era5mk>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_ivt_merra2lo).where(slopes_array_ivt_merra2lo>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_ivt_merra2cndl).where(slopes_array_ivt_merra2cndl>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_ivt_merra2ac).where(slopes_array_ivt_merra2ac>-1000)
num_sets6=num_sets6.fillna(0)

num_sets7=xr.ones_like(slopes_array_ivt_jra55gw).where(slopes_array_ivt_jra55gw>-1000)
num_sets7=num_sets7.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6+num_sets7
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,8,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,8,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_iwv_era5tp.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_era5tp=sigs_array_iwv_era5tp.where(sigs_array_iwv_era5tp<=p_final)

p_values=sigs_array_iwv_era5reid.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_era5reid=sigs_array_iwv_era5reid.where(sigs_array_iwv_era5reid<=p_final)

p_values=sigs_array_iwv_era5mk.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_era5mk=sigs_array_iwv_era5mk.where(sigs_array_iwv_era5mk<=p_final)

########


p_values=sigs_array_iwv_merra2lo.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_merra2lo=sigs_array_iwv_merra2lo.where(sigs_array_iwv_merra2lo<=p_final)

p_values=sigs_array_iwv_merra2cndl.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_merra2cndl=sigs_array_iwv_merra2cndl.where(sigs_array_iwv_merra2cndl<=p_final)

p_values=sigs_array_iwv_merra2ac.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_merra2ac=sigs_array_iwv_merra2ac.where(sigs_array_iwv_merra2ac<=p_final)

######

p_values=sigs_array_iwv_jra55gw.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_jra55gw=sigs_array_iwv_jra55gw.where(sigs_array_iwv_jra55gw<=p_final)

In [ ]:
yrs=np.arange(1980,2020,1)

mean1=iwv_concat_era5tp.mean(dim='time',skipna=True)
test1=slopes_array_iwv_era5tp/mean1*len(yrs)*100.
test1=test1.fillna(0)
mean1=iwv_concat_era5reid.mean(dim='time',skipna=True)
test2=slopes_array_iwv_era5reid/mean1*len(yrs)*100.
test2=test2.fillna(0)
mean1=iwv_concat_era5mk.mean(dim='time',skipna=True)
test3=slopes_array_iwv_era5mk/mean1*len(yrs)*100.
test3=test3.fillna(0)
mean1=iwv_concat_merra2lo.mean(dim='time',skipna=True)
test4=slopes_array_iwv_merra2lo/mean1*len(yrs)*100.
test4=test4.fillna(0)
mean1=iwv_concat_merra2cndl.mean(dim='time',skipna=True)
test5=slopes_array_iwv_merra2cndl/mean1*len(yrs)*100.
test5=test5.fillna(0)
mean1=iwv_concat_merra2ac.mean(dim='time',skipna=True)
test6=slopes_array_iwv_merra2ac/mean1*len(yrs)*100.
test6=test6.fillna(0)
mean1=fullres_iwv.mean(dim='time',skipna=True)
test7=slopes_array_iwv_jra55gw/mean1*len(yrs)*100.
test7=test7.fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets
field=test_all
field=field.where(num_sets>=2)

weights=np.cos(np.deg2rad(field.lat))

mean_rate_1=test1.weighted(weights)
mean_rate_1=mean_rate_1.mean().values.tolist()

mean_rate_2=test2.weighted(weights)
mean_rate_2=mean_rate_2.mean().values.tolist()

mean_rate_3=test3.weighted(weights)
mean_rate_3=mean_rate_3.mean().values.tolist()

mean_rate_4=test4.weighted(weights)
mean_rate_4=mean_rate_4.mean().values.tolist()

mean_rate_5=test5.weighted(weights)
mean_rate_5=mean_rate_5.mean().values.tolist()

mean_rate_6=test6.weighted(weights)
mean_rate_6=mean_rate_6.mean().values.tolist()

mean_rate_7=test7.weighted(weights)
mean_rate_7=mean_rate_7.mean().values.tolist()

list_all=[mean_rate_1,mean_rate_2,mean_rate_3,mean_rate_4,mean_rate_5,mean_rate_6,mean_rate_7]
q1=np.quantile(list_all,0.25)
q3=np.quantile(list_all,0.75)


mean_rate=field.weighted(weights)
mean_rate=mean_rate.mean().values.tolist()

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-8,9,1),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   
#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-16,18,2),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   

cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)#,ticks=np.arange(-4,5,1))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% change',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
         
#ax.text(80,92.5,'Mean change = '+str(round(mean_rate,2))+'%',fontsize=21)
ax.set_title('$IQR_{ARDTs}$ = ('+str(round(q1,1))+'%, '+str(round(q3,1))+'%)',fontsize=36)

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

#######

test1b=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp>0).fillna(0)
test2b=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid>0).fillna(0)
test3b=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk>0).fillna(0)

test4b=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo>0).fillna(0)
test5b=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl>0).fillna(0)
test6b=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac>0).fillna(0)

test7b=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b)/num_sets*100.


test1c=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp<0).fillna(0)
test2c=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid<0).fillna(0)
test3c=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk<0).fillna(0)

test4c=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo<0).fillna(0)
test5c=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl<0).fillna(0)
test6c=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac<0).fillna(0)

test7c=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c)/num_sets*100.
fieldc=test_allc

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)
test_all=test_all.where(num_sets>=2)
test_alla=test_alla.where(num_sets>=2)

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66 and test_all[X*j,X*k]<66.66 and test_alla[X*j,X*k]<66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='w',markersize=1.5,alpha=1)

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_all[X*j,X*k]>=66.66 or test_alla[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='yellow',markersize=3,alpha=1) 

plt.show()

In [ ]:
fig.savefig(dir_data+'magnitude_check_1_artmip_MEAN.png')

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_ivt_era5tp.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_era5tp=sigs_array_ivt_era5tp.where(sigs_array_ivt_era5tp<=p_final)

p_values=sigs_array_ivt_era5reid.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_era5reid=sigs_array_ivt_era5reid.where(sigs_array_ivt_era5reid<=p_final)

p_values=sigs_array_ivt_era5mk.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_era5mk=sigs_array_ivt_era5mk.where(sigs_array_ivt_era5mk<=p_final)

########


p_values=sigs_array_ivt_merra2lo.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_merra2lo=sigs_array_ivt_merra2lo.where(sigs_array_ivt_merra2lo<=p_final)

p_values=sigs_array_ivt_merra2cndl.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_merra2cndl=sigs_array_ivt_merra2cndl.where(sigs_array_ivt_merra2cndl<=p_final)

p_values=sigs_array_ivt_merra2ac.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_merra2ac=sigs_array_ivt_merra2ac.where(sigs_array_ivt_merra2ac<=p_final)

######

p_values=sigs_array_ivt_jra55gw.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_jra55gw=sigs_array_ivt_jra55gw.where(sigs_array_ivt_jra55gw<=p_final)

In [ ]:
mean1=ivt_concat_era5tp.mean(dim='time',skipna=True)
test1=slopes_array_ivt_era5tp/mean1*len(yrs)*100.
test1=test1.fillna(0)
mean1=ivt_concat_era5reid.mean(dim='time',skipna=True)
test2=slopes_array_ivt_era5reid/mean1*len(yrs)*100.
test2=test2.fillna(0)
mean1=ivt_concat_era5mk.mean(dim='time',skipna=True)
test3=slopes_array_ivt_era5mk/mean1*len(yrs)*100.
test3=test3.fillna(0)
mean1=ivt_concat_merra2lo.mean(dim='time',skipna=True)
test4=slopes_array_ivt_merra2lo/mean1*len(yrs)*100.
test4=test4.fillna(0)
mean1=ivt_concat_merra2cndl.mean(dim='time',skipna=True)
test5=slopes_array_ivt_merra2cndl/mean1*len(yrs)*100.
test5=test5.fillna(0)
mean1=ivt_concat_merra2ac.mean(dim='time',skipna=True)
test6=slopes_array_ivt_merra2ac/mean1*len(yrs)*100.
test6=test6.fillna(0)
mean1=fullres_ivt.mean(dim='time',skipna=True)
test7=slopes_array_ivt_jra55gw/mean1*len(yrs)*100.
test7=test7.fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets
field=test_all
field=field.where(num_sets>=2)

weights=np.cos(np.deg2rad(field.lat))

mean_rate_1=test1.weighted(weights)
mean_rate_1=mean_rate_1.mean().values.tolist()

mean_rate_2=test2.weighted(weights)
mean_rate_2=mean_rate_2.mean().values.tolist()

mean_rate_3=test3.weighted(weights)
mean_rate_3=mean_rate_3.mean().values.tolist()

mean_rate_4=test4.weighted(weights)
mean_rate_4=mean_rate_4.mean().values.tolist()

mean_rate_5=test5.weighted(weights)
mean_rate_5=mean_rate_5.mean().values.tolist()

mean_rate_6=test6.weighted(weights)
mean_rate_6=mean_rate_6.mean().values.tolist()

mean_rate_7=test7.weighted(weights)
mean_rate_7=mean_rate_7.mean().values.tolist()

list_all=[mean_rate_1,mean_rate_2,mean_rate_3,mean_rate_4,mean_rate_5,mean_rate_6,mean_rate_7]
q1=np.quantile(list_all,0.25)
q3=np.quantile(list_all,0.75)

mean_rate=field.weighted(weights)
mean_rate=mean_rate.mean().values.tolist()

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-8,9,1),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   
#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-16,18,2),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   

cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)#,ticks=np.arange(-4,5,1))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% change',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
         
#ax.text(80,92.5,'Mean change = '+str(round(mean_rate,2))+'%',fontsize=21)
ax.set_title('$IQR_{ARDTs}$ = ('+str(round(q1,1))+'%, '+str(round(q3,1))+'%)',fontsize=36)


test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

#######

test1b=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp>0).fillna(0)
test2b=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid>0).fillna(0)
test3b=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk>0).fillna(0)

test4b=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo>0).fillna(0)
test5b=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl>0).fillna(0)
test6b=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac>0).fillna(0)

test7b=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b)/num_sets*100.


test1c=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp<0).fillna(0)
test2c=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid<0).fillna(0)
test3c=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk<0).fillna(0)

test4c=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo<0).fillna(0)
test5c=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl<0).fillna(0)
test6c=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac<0).fillna(0)

test7c=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c)/num_sets*100.
fieldc=test_allc

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)
test_all=test_all.where(num_sets>=2)
test_alla=test_alla.where(num_sets>=2)

plt.show()

In [ ]:
fig.savefig(dir_data+'magnitude_check_2_artmip_MEAN.png')

In [ ]:
num_sets1=xr.ones_like(slopes_array_ivt_era5tp).where(slopes_array_s_era5tp>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_ivt_era5reid).where(slopes_array_s_era5reid>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_ivt_era5mk).where(slopes_array_s_era5mk>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_ivt_merra2lo).where(slopes_array_s_merra2lo>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_ivt_merra2cndl).where(slopes_array_s_merra2cndl>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_ivt_merra2ac).where(slopes_array_s_merra2ac>-1000)
num_sets6=num_sets6.fillna(0)

num_sets7=xr.ones_like(slopes_array_ivt_jra55gw).where(slopes_array_s_jra55gw>-1000)
num_sets7=num_sets7.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6+num_sets7
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,8,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,8,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_s_era5tp.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_era5tp=sigs_array_s_era5tp.where(sigs_array_s_era5tp<=p_final)

p_values=sigs_array_s_era5reid.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_era5reid=sigs_array_s_era5reid.where(sigs_array_s_era5reid<=p_final)

p_values=sigs_array_s_era5mk.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_era5mk=sigs_array_s_era5mk.where(sigs_array_s_era5mk<=p_final)

########


p_values=sigs_array_s_merra2lo.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_merra2lo=sigs_array_s_merra2lo.where(sigs_array_s_merra2lo<=p_final)

p_values=sigs_array_s_merra2cndl.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_merra2cndl=sigs_array_s_merra2cndl.where(sigs_array_s_merra2cndl<=p_final)

p_values=sigs_array_s_merra2ac.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_merra2ac=sigs_array_s_merra2ac.where(sigs_array_s_merra2ac<=p_final)

######

p_values=sigs_array_s_jra55gw.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_jra55gw=sigs_array_s_jra55gw.where(sigs_array_s_jra55gw<=p_final)

In [ ]:
mean1=s_concat_era5tp.mean(dim='time',skipna=True)
test1=slopes_array_s_era5tp/mean1*len(yrs)*100.
test1=test1.fillna(0)
mean1=s_concat_era5reid.mean(dim='time',skipna=True)
test2=slopes_array_s_era5reid/mean1*len(yrs)*100.
test2=test2.fillna(0)
mean1=s_concat_era5mk.mean(dim='time',skipna=True)
test3=slopes_array_s_era5mk/mean1*len(yrs)*100.
test3=test3.fillna(0)
mean1=s_concat_merra2lo.mean(dim='time',skipna=True)
test4=slopes_array_s_merra2lo/mean1*len(yrs)*100.
test4=test4.fillna(0)
mean1=s_concat_merra2cndl.mean(dim='time',skipna=True)
test5=slopes_array_s_merra2cndl/mean1*len(yrs)*100.
test5=test5.fillna(0)
mean1=s_concat_merra2ac.mean(dim='time',skipna=True)
test6=slopes_array_s_merra2ac/mean1*len(yrs)*100.
test6=test6.fillna(0)
mean1=fullres_s.mean(dim='time',skipna=True)
test7=slopes_array_s_jra55gw/mean1*len(yrs)*100.
test7=test7.fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets
field=test_all
field=field.where(num_sets>=2)

weights=np.cos(np.deg2rad(field.lat))

mean_rate_1=test1.weighted(weights)
mean_rate_1=mean_rate_1.mean().values.tolist()

mean_rate_2=test2.weighted(weights)
mean_rate_2=mean_rate_2.mean().values.tolist()

mean_rate_3=test3.weighted(weights)
mean_rate_3=mean_rate_3.mean().values.tolist()

mean_rate_4=test4.weighted(weights)
mean_rate_4=mean_rate_4.mean().values.tolist()

mean_rate_5=test5.weighted(weights)
mean_rate_5=mean_rate_5.mean().values.tolist()

mean_rate_6=test6.weighted(weights)
mean_rate_6=mean_rate_6.mean().values.tolist()

mean_rate_7=test7.weighted(weights)
mean_rate_7=mean_rate_7.mean().values.tolist()

list_all=[mean_rate_1,mean_rate_2,mean_rate_3,mean_rate_4,mean_rate_5,mean_rate_6,mean_rate_7]
q1=np.quantile(list_all,0.25)
q3=np.quantile(list_all,0.75)

mean_rate=field.weighted(weights)
mean_rate=mean_rate.mean().values.tolist()

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-8,9,1),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   
#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-16,18,2),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   

cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)#,ticks=np.arange(-4,5,1))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% change',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
         
#ax.text(80,92.5,'Mean change = '+str(round(mean_rate,2))+'%',fontsize=21)
ax.set_title('$IQR_{ARDTs}$ = ('+str(round(q1,1))+'%, '+str(round(q3,1))+'%)',fontsize=36)

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_s_era5tp<=0.05).where(slopes_array_s_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_s_era5reid<=0.05).where(slopes_array_s_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_s_era5mk<=0.05).where(slopes_array_s_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_s_merra2lo<=0.05).where(slopes_array_s_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_s_merra2ac<=0.05).where(slopes_array_s_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_s_jra55gw<=0.05).where(slopes_array_s_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_s_era5tp<=0.05).where(slopes_array_s_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_s_era5reid<=0.05).where(slopes_array_s_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_s_era5mk<=0.05).where(slopes_array_s_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_s_merra2lo<=0.05).where(slopes_array_s_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_s_merra2ac<=0.05).where(slopes_array_s_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_s_jra55gw<=0.05).where(slopes_array_s_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

#######

test1b=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_s_era5tp<=0.05).where(slopes_array_s_era5tp>0).fillna(0)
test2b=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_s_era5reid<=0.05).where(slopes_array_s_era5reid>0).fillna(0)
test3b=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_s_era5mk<=0.05).where(slopes_array_s_era5mk>0).fillna(0)

test4b=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_s_merra2lo<=0.05).where(slopes_array_s_merra2lo>0).fillna(0)
test5b=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl>0).fillna(0)
test6b=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_s_merra2ac<=0.05).where(slopes_array_s_merra2ac>0).fillna(0)

test7b=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_s_jra55gw<=0.05).where(slopes_array_s_jra55gw>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b)/num_sets*100.


test1c=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_s_era5tp<=0.05).where(slopes_array_s_era5tp<0).fillna(0)
test2c=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_s_era5reid<=0.05).where(slopes_array_s_era5reid<0).fillna(0)
test3c=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_s_era5mk<=0.05).where(slopes_array_s_era5mk<0).fillna(0)

test4c=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_s_merra2lo<=0.05).where(slopes_array_s_merra2lo<0).fillna(0)
test5c=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl<0).fillna(0)
test6c=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_s_merra2ac<=0.05).where(slopes_array_s_merra2ac<0).fillna(0)

test7c=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_s_jra55gw<=0.05).where(slopes_array_s_jra55gw<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c)/num_sets*100.
fieldc=test_allc

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)
test_all=test_all.where(num_sets>=2)
test_alla=test_alla.where(num_sets>=2)

plt.show()

In [ ]:
fig.savefig(dir_data+'magnitude_check_3_artmip_MEAN.png')

In [ ]:
num_sets1=xr.ones_like(slopes_array_ivt_era5tp).where(slopes_array_mfc_era5tp>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_ivt_era5reid).where(slopes_array_mfc_era5reid>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_ivt_era5mk).where(slopes_array_mfc_era5mk>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_ivt_merra2lo).where(slopes_array_mfc_merra2lo>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_ivt_merra2cndl).where(slopes_array_mfc_merra2cndl>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_ivt_merra2ac).where(slopes_array_mfc_merra2ac>-1000)
num_sets6=num_sets6.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,8,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,8,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_mfc_era5tp.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_era5tp=sigs_array_mfc_era5tp.where(sigs_array_mfc_era5tp<=p_final)

p_values=sigs_array_mfc_era5reid.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_era5reid=sigs_array_mfc_era5reid.where(sigs_array_mfc_era5reid<=p_final)

p_values=sigs_array_mfc_era5mk.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_era5mk=sigs_array_mfc_era5mk.where(sigs_array_mfc_era5mk<=p_final)

########


p_values=sigs_array_mfc_merra2lo.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_merra2lo=sigs_array_mfc_merra2lo.where(sigs_array_mfc_merra2lo<=p_final)

p_values=sigs_array_mfc_merra2cndl.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_merra2cndl=sigs_array_mfc_merra2cndl.where(sigs_array_mfc_merra2cndl<=p_final)

p_values=sigs_array_mfc_merra2ac.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_merra2ac=sigs_array_mfc_merra2ac.where(sigs_array_mfc_merra2ac<=p_final)

In [ ]:
print(yrs)
if 1==0:
    mean1=mfc_concat_era5tp.mean(dim='time',skipna=True)
    test1=slopes_array_mfc_era5tp*len(yrs)
    test1=test1.fillna(0)/1e-5
    mean1=mfc_concat_era5reid.mean(dim='time',skipna=True)
    test2=slopes_array_mfc_era5reid*len(yrs)
    test2=test2.fillna(0)/1e-5
    mean1=mfc_concat_era5mk.mean(dim='time',skipna=True)
    test3=slopes_array_mfc_era5mk*len(yrs)
    test3=test3.fillna(0)/1e-5
    mean1=mfc_concat_merra2lo.mean(dim='time',skipna=True)
    test4=slopes_array_mfc_merra2lo*len(yrs)
    test4=test4.fillna(0)/1e-5
    mean1=mfc_concat_merra2cndl.mean(dim='time',skipna=True)
    test5=slopes_array_mfc_merra2cndl*len(yrs)
    test5=test5.fillna(0)/1e-5
    mean1=mfc_concat_merra2ac.mean(dim='time',skipna=True)
    test6=slopes_array_mfc_merra2ac*len(yrs)
    test6=test6.fillna(0)/1e-5

if 1==1:
    mean1=mfc_concat_era5tp.mean(dim='time',skipna=True)
    test1=slopes_array_mfc_era5tp*len(yrs)/mean1*100.
    test1=test1.fillna(0)
    #test1=test1.fillna(0)/1e-5
    mean1=mfc_concat_era5reid.mean(dim='time',skipna=True)
    test2=slopes_array_mfc_era5reid*len(yrs)/mean1*100.
    test2=test2.fillna(0)
    #test2=test2.fillna(0)/1e-5
    mean1=mfc_concat_era5mk.mean(dim='time',skipna=True)
    test3=slopes_array_mfc_era5mk*len(yrs)/mean1*100.
    test3=test3.fillna(0)#/1e-5
    mean1=mfc_concat_merra2lo.mean(dim='time',skipna=True)
    test4=slopes_array_mfc_merra2lo*len(yrs)/mean1*100.
    test4=test4.fillna(0)#/1e-5
    mean1=mfc_concat_merra2cndl.mean(dim='time',skipna=True)
    test5=slopes_array_mfc_merra2cndl*len(yrs)/mean1*100.
    test5=test5.fillna(0)#/1e-5
    mean1=mfc_concat_merra2ac.mean(dim='time',skipna=True)
    test6=slopes_array_mfc_merra2ac*len(yrs)/mean1*100.
    test6=test6.fillna(0)#/1e-5

test_all=(test1+test2+test3+test4+test5+test6)/num_sets
field=test_all
field=field.where(num_sets>=2)

#field=test4

weights=np.cos(np.deg2rad(field.lat))

mean_rate_1=test1.weighted(weights)
mean_rate_1=mean_rate_1.mean().values.tolist()

mean_rate_2=test2.weighted(weights)
mean_rate_2=mean_rate_2.mean().values.tolist()

mean_rate_3=test3.weighted(weights)
mean_rate_3=mean_rate_3.mean().values.tolist()

mean_rate_4=test4.weighted(weights)
mean_rate_4=mean_rate_4.mean().values.tolist()

mean_rate_5=test5.weighted(weights)
mean_rate_5=mean_rate_5.mean().values.tolist()

mean_rate_6=test6.weighted(weights)
mean_rate_6=mean_rate_6.mean().values.tolist()

list_all=[mean_rate_1,mean_rate_2,mean_rate_3,mean_rate_4,mean_rate_5,mean_rate_6]
q1=np.quantile(list_all,0.25)
q3=np.quantile(list_all,0.75)

#mean_rate=field.weighted(weights)
#mean_rate=mean_rate.mean().values.tolist()

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-5,5.5,0.5),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   
#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-100,110,10),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-40,44,4),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   

cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)#,ticks=np.arange(-4,5,1))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
#cbar.set_label('10$^{-5}$ kg m$^{-2}$ s$^{-1}$',fontsize=36,labelpad=10)
cbar.set_label('% change',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
         
#ax.text(80,92.5,'Mean change = '+str(round(mean_rate,2))+'%',fontsize=21)
#ax.set_title('$IQR_{ARDTs}$ = ('+str(round(q1,1))+', '+str(round(q3,1))+') x 10$^{-5}$ kg m$^{-2}$ s$^{-1}$',fontsize=36)
ax.set_title('$IQR_{ARDTs}$ = ('+str(round(q1,1))+'%, '+str(round(q3,1))+'%)',fontsize=36)



test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

#######

test1b=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp>0).fillna(0)
test2b=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid>0).fillna(0)
test3b=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk>0).fillna(0)

test4b=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo>0).fillna(0)
test5b=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl>0).fillna(0)
test6b=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b)/num_sets*100.


test1c=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp<0).fillna(0)
test2c=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid<0).fillna(0)
test3c=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk<0).fillna(0)

test4c=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo<0).fillna(0)
test5c=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl<0).fillna(0)
test6c=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c)/num_sets*100.
fieldc=test_allc

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)
test_all=test_all.where(num_sets>=2)
test_alla=test_alla.where(num_sets>=2)

plt.show()

In [ ]:
fig.savefig(dir_data+'magnitude_check_4_artmip_MAXa.png')

In [ ]:
test=mfc_concat_merra2lo

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,test[10,:,:]*1e4,levels=np.arange(-8,9,1),cmap=cmap_new,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)#,ticks=np.arange(-4,5,1))

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% change',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
         
#ax.text(80,92.5,'Mean change = '+str(round(mean_rate,2))+'%',fontsize=21)
ax.set_title('Mean change = '+str(round(mean_rate,2))+'%',fontsize=36)

plt.show()